# Hyperparam optimisation

In [1]:
%load_ext autoreload
%autoreload 2

# %matplotlib widget

In [2]:
import pandas as pd
import warnings
import seaborn as sns

import optuna
from optuna.samplers import TPESampler
from optuna.visualization import (
    plot_optimization_history,
    plot_contour,
)

import numpy as np
import matplotlib.pyplot as plt

from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error

from poly2.xgb import HyperparamsObj, check_default_model

C:\Users\user\anaconda3\envs\poly2\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
MODEL = 'all'
# MODEL = 'Y10'
# MODEL = 'cumulative'
# MODEL = 'asymp'

# Load data

In [4]:
if MODEL=='Y10':
    df = (
        pd.read_csv('../outputs/combined/processed_scan_all.csv')
        .loc[lambda x: x.year==10]
        .reset_index(drop=True)
    )
    X = df.drop(['year', 'best_dose'], axis=1)
    
else:
    df = pd.read_csv(f'../outputs/combined/processed_scan_{MODEL}.csv')
    X = df.drop(['best_dose'], axis=1)

y = df.loc[:, ['run', 'best_dose']]

df.head(2)

,run,year,best_dose,mu,b,asymp,dec_rate,m_prop,m_scale,ME_mean
0,0,1,1.0,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,0.397237
1,0,2,1.0,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,0.397237


Get data (by run)

In [5]:
X_cv = X.loc[lambda x: (x.run<8000)].drop('run', axis=1)
y_cv = y.loc[lambda x: (x.run<8000)].drop('run', axis=1)

X_test = X.loc[lambda x: (x.run>=8000)].drop('run', axis=1)
y_test = np.array(y.loc[lambda x: (x.run>=8000)].drop('run', axis=1))

X_cv.head(2)

,year,mu,b,asymp,dec_rate,m_prop,m_scale,ME_mean
0,1,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,0.397237
1,2,17.879734,2.744068,0.602763,1.786355,0.703574,1.95789,0.397237


# Find optimal hyperparams

In [6]:
np.random.seed(0)

optuna.logging.set_verbosity(0)

# ignore warning about Int64Index
warnings.simplefilter(action='ignore', category=FutureWarning)

How do the default params do?

In [ ]:
%%time
score = check_default_model(X_cv, y_cv)
score

2it [00:07,  3.71s/it]

Score obtained: 

- all: `XX`
- Y10: `XX`
- asymp: `XX`
- cumulative: `XX`

## Hyperparam opt

In [ ]:
obj = HyperparamsObj(X_cv, y_cv)

sampler = TPESampler(seed=10)
study = optuna.create_study(sampler=sampler)

NB takes about 17 hours

In [ ]:
%%time
study.optimize(obj, n_trials=10)
study.best_value

In [ ]:
%%time
study.optimize(obj, n_trials=90)
study.best_value

In [ ]:
%%time
study.optimize(obj, n_trials=100)
study.best_value

In [ ]:
%%time
study.optimize(obj, n_trials=100)
study.best_value

Score obtained: 

- all: `XX`
- Y10: `XX`
- asymp: `XX`
- cumulative: `XX`

## Optimisation plots

In [ ]:
f = plot_optimization_history(study)
f.update_layout(height=500)

In [ ]:
f = plot_contour(study)
f.update_layout(height=800)

## Check performance of best model

Check model doesn't drastically degrade on completely unseen data

#### Train set:
- all: `XX`
- Y10: `XX`
- asymp: `XX`
- cumulative: `XX`

#### Test set:
- all: `XX`
- Y10: `XX`
- asymp: `XX`
- cumulative: `XX`

In [ ]:
best_pars = study.best_params
best_pars

In [ ]:
load_best_pars = False

if load_best_pars:
    best_pars = pd.read_csv(f'../outputs/best_XGB_hyperparams_{MODEL}.csv').iloc[0].to_dict()
    
    best_pars['max_depth'] = int(best_pars['max_depth'])
    best_pars['n_estimators'] = int(best_pars['n_estimators'])
    # best_pars['objective'] = 'reg:pseudohubererror'
    
else:
    # save
    pd.DataFrame(best_pars, index=[0]).to_csv(f'../outputs/best_XGB_hyperparams_{MODEL}.csv', index=False)

### Performance on test set:

In [ ]:
%%time

best_model = XGBRegressor(**best_pars).fit(X_cv, y_cv)

y_p = best_model.predict(X_test)

rmse = mean_squared_error(y_p, y_test, squared=False)

rmse

### Performance on training set:

In [ ]:
y_pt = best_model.predict(X_cv)

rmse = mean_squared_error(y_pt, y_cv, squared=False)

rmse

### Plot performance (rough)

In [ ]:
f, ax = plt.subplots()

(
    pd.DataFrame()
    .assign(
        model = y_p,
        data = y_test,
    )
    .sort_values('model')
    .assign(data = lambda x: x.data + np.random.normal(0, 0.01, x.shape[0]))
    .plot
    .scatter(
        x='data',
        y='model',
        alpha=0.1,
        ax=ax
    )
)

ax.scatter(np.arange(0.1, 1.1, 0.1), np.arange(0.1, 1.1, 0.1), c='r')

ax.set_xlim([0,1.08])
ax.set_ylim([-0.2, 1.3])

### Performance of default model on test set:

In [ ]:
# model = XGBRegressor().fit(X_train_val, y_train_val)
model = XGBRegressor().fit(X_cv, y_cv)

y_pdm = model.predict(X_test)

rmse = mean_squared_error(y_pdm, y_test, squared=False)

rmse

## Save best model

In [ ]:
best_model.save_model(f'xgb_{MODEL}.json')